# Start the RIAK Server for the CICI SAFE vSDX

#### Configure the Envrionment

In [1]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
#export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
#export OS_REGION_NAME='CHI@TACC'
#export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
# NODE_RESERVATION_ID must be for Skylake nodes
#export RIAK_RESERVATION_ID='8882c0ea-e92a-404e-8637-73b4c38f4101'


#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom

export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info

export OS_REGION_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_REGION_NAME | cut -d " " -f 2`
export OS_PROJECT_NAME=`cat $UNIVERSAL_INFO_FILE | grep OS_PROJECT_NAME | cut -d " " -f 2`
echo OS_REGION_NAME $OS_REGION_NAME
echo OS_PROJECT_NAME $OS_PROJECT_NAME


OS_REGION_NAME CHI@UC
OS_PROJECT_NAME CH-816532


In [2]:
#Clear RIAK info
echo "" > $RIAK_INFO_FILE

#### Create Lease (if necessary)

In [3]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation
    # --start-date "2019-02-10 17:50" \
    # --end-date "2019-02-11 12:00" \
    START_DATE=`date -d "+2 min" +'%F %H:%M'`
    END_DATE=`date -d "+1 day" +'%F %H:%M'`
    LEASE_NAME=${USERNAME}"-riak-lease"
    echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE

    NODE_MIN=1
    NODE_MAX=1

    NODE_TYPE='compute_haswell'
    #NODE_TYPE='compute_skylake'

    echo Creating Lease ${LEASE_NAME}
    blazar lease-create \
       --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
       --start-date "${START_DATE}" \
       --end-date "${END_DATE}" \
       ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $RIAK_INFO_FILE

pruth-cici-universal-lease2
LEASE_NAME: pruth-cici-universal-lease2

LEASE_NAME pruth-cici-universal-lease2


In [4]:
LEASE_NAME=`cat $RIAK_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $RIAK_INFO_FILE

cat $RIAK_INFO_FILE
#blazar lease-show ${RESERVATION_ID}

RESERVATION_ID 480a3fde-d50b-40d8-b48b-781729342b20
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
RESERVATION_STATUS ACTIVE

LEASE_NAME pruth-cici-universal-lease2
RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc


#### Start RIAK Server

In [5]:
#RESERVATION_RESOURCE_ID=`cat $RIAK_RESERVATION_FILE`
RESERVATION_RESOURCE_ID=`cat $RIAK_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
echo  RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

RIAK_NETWORK='sharednet1'
RIAK_NODE_NAME=${USERNAME}-riak
RIAK_STACK_NAME=${USERNAME}-riak

echo RIAK_NODE_NAME $RIAK_NODE_NAME >> $RIAK_INFO_FILE
echo RIAK_STACK_NAME $RIAK_STACK_NAME >> $RIAK_INFO_FILE

RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc


In [6]:
#IMAGE='CC-CentOS7'
IMAGE='CICI-SAFE.v1'
RESERVATION_RESOURCE_ID=`cat $RIAK_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
echo  RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image $IMAGE \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${RESERVATION_RESOURCE_ID} \
                        --security-group default  \
                        --nic net-id=${RIAK_NETWORK} \
                        --wait \
                        ${RIAK_NODE_NAME}

echo Server creation complete! 
echo ${RIAK_NODE_NAME} is `openstack server show --format value -c status ${RIAK_NODE_NAME}`


RESERVATION_RESOURCE_ID 6f73c9e3-fbdf-4019-b426-6e54b99366cc
Creating servers... This will take several minutes!

+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         | nova                                   |
| OS-EXT-SRV-ATTR:host                | admin01-ironic                         |
| OS-EXT-SRV-ATTR:hypervisor_hostname | 60f65717-a665-4b0d-af50-1ac46e77e587   |
| OS-EXT-SRV-ATTR:instance_name       | instance-00008368                      |
| OS-EXT-STS:power_state              | Running                                |
| OS-EXT-STS:task_state               | None                                   |
| OS-EXT-STS:vm_state                 | active                              

In [7]:
# Wait until server is ready before continuing
server_status=""
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   echo Getting status for $RIAK_NODE_NAME
   server_status=$(openstack server list --format value -c Status --name "$RIAK_NODE_NAME")
   echo status $server_status
done
echo Status is $server_status

Getting status for pruth-riak
status ACTIVE
Status is ACTIVE


In [8]:
# Request a public floating IP (in the 'public' network)
RIAK_PUBLIC_IP=$(openstack floating ip create public --format value -c floating_ip_address)

echo RIAK_PUBLIC_IP $RIAK_PUBLIC_IP

# Assign a public floating IP to $USER-first-server
openstack server add floating ip "$RIAK_NODE_NAME" "$RIAK_PUBLIC_IP"
echo RIAK_PUBLIC_IP $RIAK_PUBLIC_IP >> $RIAK_INFO_FILE

RIAK_PUBLIC_IP 192.5.87.143


In [9]:
#echo Creating RIAK server. This will take several minutes!
#openstack stack create  --max-width 80 \
#        --template riak.yaml \
#        --parameter "key_name=${SSH_KEY_NAME}" \
#        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
#        --parameter "network_name=${RIAK_NETWORK}" \
#        --parameter "riak_node_name=${RIAK_NODE_NAME}" \
#        --wait \
#        ${RIAK_STACK_NAME}

In [10]:
#echo RIAK server creation complete! 
#echo ${RIAK_NODE_NAME} status `openstack server show  --format value -c status ${RIAK_NODE_NAME}`
#
#RIAK_PUBLIC_IP=`openstack server show  --format value -c addresses ${RIAK_NODE_NAME} | cut -d " " -f 2`
#echo The RIAK se public IP is $RIAK_PUBLIC_IP
#echo RIAK_PUBLIC_IP $RIAK_PUBLIC_IP >> $RIAK_INFO_FILE


In [11]:
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`
echo Trying ssh to $RIAK_PUBLIC_IP
#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${RIAK_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo RIAK node ready for ssh

Trying ssh to 192.5.87.143
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
Node not ready for ssh. retrying in 5 seconds.
RIAK node ready for ssh


In [12]:
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

REMOTE_SCRIPT=/home/cc/script.sh
LOCAL_SCRIPT=${WORKING_DIR}/riak_script.sh

cat << EOF > $LOCAL_SCRIPT
#!/bin/bash
#sudo yum install -y docker vim mlocate
sudo systemctl start docker

#docker pull yaoyj11/riakimg
sudo docker run -i -t  -d -p 2122:2122 -p 8098:8098 -p 8087:8087 -h riakserver --name riakserver yaoyj11/riakimg
sudo docker exec -itd riakserver sudo riak start
sudo docker exec -itd riakserver sudo riak-admin bucket-type activate  safesets
sudo docker exec -itd riakserver sudo riak-admin bucket-type update safesets '{"props":{"allow_mult":false}}'
echo Boot Script Done!

EOF

echo Copying script to cc@$RIAK_PUBLIC_IP
scp -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    $LOCAL_SCRIPT \
    cc@${RIAK_PUBLIC_IP}:${REMOTE_SCRIPT}
echo Done

echo Running script
ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${RIAK_PUBLIC_IP} "chmod +x "$REMOTE_SCRIPT" && "$REMOTE_SCRIPT

echo Done

Copying script to cc@192.5.87.143
Done
Running script
f00c183d78b65f2eaf0c72a04c3a5b1d1aef8c2f94eb842759eceaedf8e7dc0c
Boot Script Done!
Done


#### Clean Up RIAK

In [ ]:
RIAK_STACK_NAME=`cat $RIAK_INFO_FILE | grep RIAK_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $RIAK_STACK_NAME

LEASE_NAME=`cat $RIAK_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $RIAK_INFO_FILE
mv $RIAK_INFO_FILE ${RIAK_INFO_FILE}.old